In [15]:
%reset -f
import numpy as np
import sys
from datetime import datetime
import os

In [16]:
output_log_file = "experiment_3_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

In [17]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [18]:
attack_rate_list = [0]

In [19]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """

    # Part 1: Extract hardness values from all edges and append them
    # into one numpy array
    # Calculate hardness values for each edge 
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize variables for max weight loop
        weights = []
        # Collect all weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        # Initialize variables for min weight loop
        min_weights = []
        # Collect all weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        # Convert weights to probabilities
        # We could take max_weight or min_weight here
        # hardness.append(np.exp(-max_weight))

        # Important: We use min_weight here because of the following reason:
        # Since the formula to calculate hardness in R is hardness = exp(-weight)
        # taking the minimum weight will give us the maximum hardness
        # which translates to the path being EASIEST to traverse.
        # Yes hardness of 1 means path is trivial, hardness 0 means path is impossible
        hardness.append(np.exp(-min_weight))

    
    
    hardness = np.array(hardness)

    # print(f'Hardness: {hardness}')

    
    ## Part 2: Based on the extracted hardness values
    ## Calculate Movement Probabilities

    # We calculate two things:
    # 1. Probability of reaching each node (accumulating hardness along the way)
    # Example: if hardness = [0.8, 0.6, 0.4]
    # Then cumprod gives us: [0.8, 0.8*0.6, 0.8*0.6*0.4]
    # Final cumulative_probs = [1.0, 0.8, 0.48, 0.192]
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))

    # 2. Probability of stopping at each node (based on the next edge's hardness)
    stop_probs = np.concatenate((1 - hardness, [1.0]))

    ## Part 3: Generate Final Distribution
    # Combine reaching and stopping probabilities to get probability of stopping at each node
    # Example calculation with above values:
    # Node0: 1.0 * 0.2 = 0.2    (20% chance of stopping at start)
    # Node1: 0.8 * 0.4 = 0.32   (32% chance of stopping at Node1)
    # Node2: 0.48 * 0.6 = 0.288 (28.8% chance of stopping at Node2)
    # Node3: 0.192 * 1.0 = 0.192 (19.2% chance of reaching final node)
    pdf = cumulative_probs * stop_probs

    # Handle case where probabilities are essentially zero
    if pdf.sum() < 1e-15:
        pdf = np.full_like(pdf, 1e-7)

    # Normalize to ensure probabilities sum to 1
    # print(f"This is the final pdf that is returned in the end: {pdf / pdf.sum()}")
    return pdf / pdf.sum()

In [20]:
%run heuristic_defense.ipynb

In [21]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())